In [1]:
import pandas as pd
from datasets import Dataset, ClassLabel, Sequence
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline
)
import numpy as np
from seqeval.metrics import classification_report, f1_score, accuracy_score
from datasets import Dataset
from itertools import chain

c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data = pd.read_csv("data.csv")

In [3]:
#data.to_xml()

In [ ]:
training_samples = [
    # 1
    {
        "tokens": ["Quais","são","os","5","ativos","de","renda","fixa","com","risco","baixo","e",
                   "rentabilidade","acima","de","10","%","ao","ano",",","atrelados","ao","CDI","?"],
        "ner_tags": ["O","O","O","B-QUANTIDADE","O","O","B-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "O","O","B-TAXA_ATRELADA","O"]
    },
    # 2
    {
        "tokens": ["Liste","3","ações","de","risco","alto","com","rentabilidade","superior","a",
                   "20","%","nos","últimos","12","meses","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","O","B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O"]
    },
    # 3
    {
        "tokens": ["Mostre","2","fundos","imobiliários","atrelados","ao","IPCA","que","apresentem",
                   "rentabilidade","maior","que","12","%","e","risco","moderado","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","O","O","B-TAXA_ATRELADA","O","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "O","B-RISCO","I-RISCO","O"]
    },
    # 4
    {
        "tokens": ["Quais","4","títulos","com","o","nome","‘","Tesouro","Selic","2027","’","possuem",
                   "risco","muito","baixo","e","rentabilidade","de","cerca","de","9","%","ao","ano","?"],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","O","O","O","O",
                     "B-NOME_ATIVO","I-NOME_ATIVO","I-NOME_ATIVO","O","O",
                     "B-RISCO","I-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O"]
    },
    # 5
    {
        "tokens": ["Quero","ver","6","ETFs","de","renda","variável","cuja","taxa","seja","atrelada","ao","dólar",",",
                   "com","rentabilidade","acima","de","18","%","e","risco","alto","."],
        "ner_tags": ["O","O","B-QUANTIDADE","B-TIPO_ATIVO","O","I-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "O","O","O","O","B-TAXA_ATRELADA","O",
                     "O","B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "O","B-RISCO","I-RISCO","O"]
    },
    # 6
    {
        "tokens": ["Mostre","os","3","ativos","de","renda","fixa","com","a","menor","rentabilidade","possível",",",
                   "risco","baixíssimo","e","taxa","indexada","à","Selic","."],
        "ner_tags": ["O","O","B-QUANTIDADE","O","O","B-TIPO_ATIVO","I-TIPO_ATIVO","O","O",
                     "O","B-RENTABILIDADE","I-RENTABILIDADE","O",
                     "B-RISCO","I-RISCO","O",
                     "O","O","O","B-TAXA_ATRELADA","O"]
    },
    # 7
    {
        "tokens": ["Liste","1","debênture","de","risco","médio",",","atrelada","ao","CDI",",","cuja",
                   "rentabilidade","supere","11","%","ao","ano","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","O","B-RISCO","I-RISCO","O",
                     "O","O","B-TAXA_ATRELADA","O","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O"]
    },
    # 8
    {
        "tokens": ["Quais","são","os","7","CDBs","com","rentabilidade","igual","ou","superior","a","13","%","ao","ano",",",
                   "risco","moderado","e","taxa","ligada","ao","CDI","?"],
        "ner_tags": ["O","O","O","B-QUANTIDADE","B-TIPO_ATIVO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "B-RISCO","I-RISCO","O","O","O","B-TAXA_ATRELADA","O"]
    },
    # 9
    {
        "tokens": ["Traga","2","ações","de","tecnologia","de","risco","alto",",","rentabilidade","acima","de",
                   "25","%","e","taxa","atrelada","ao","IPCA","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "O","O","O","B-TAXA_ATRELADA","O"]
    },
    # 10
    {
        "tokens": ["Quais","10","ativos","de","renda","variável","com","o","nome","‘","Petrobras","PN","’","têm",
                   "risco","médio","e","rentabilidade","de","pelo","menos","15","%","ao","ano","?"],
        "ner_tags": ["O","B-QUANTIDADE","O","O","B-TIPO_ATIVO","I-TIPO_ATIVO","O","O","O","O",
                     "B-NOME_ATIVO","I-NOME_ATIVO","O","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O"]
    },
]


In [5]:
validation_samples = [
    # 11 ─────────────────────────────────────────────────────────────────────
    {
        "tokens": ["Me","mostre","8","títulos","de","renda","fixa","com","risco","alto",",",
                   "rentabilidade","acima","de","14","%","e","taxa","atrelada","ao","IPCA","."],
        "ner_tags": ["O","O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","I-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "O","O","O","O","B-TAXA_ATRELADA","O"]
    },
    # 12 ─────────────────────────────────────────────────────────────────────
    {
        "tokens": ["Quero","2","debêntures","prefixadas","de","risco","baixo",",","rentabilidade","mínima","de",
                   "10","%","ao","ano","e","indexadas","à","Selic","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE",
                     "I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "O","O","B-TAXA_ATRELADA","O"]
    },
    # 13 ─────────────────────────────────────────────────────────────────────
    {
        "tokens": ["Liste","5","CDBs","com","risco","médio",",","taxas","ligadas","ao","CDI","e",
                   "rentabilidade","superior","a","12","%","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "O","O","O","B-TAXA_ATRELADA","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O"]
    },
    # 14 ─────────────────────────────────────────────────────────────────────
    {
        "tokens": ["Quais","3","ações","de","saúde","oferecem","rentabilidade","acima","de","30","%","e","risco",
                   "alto",",","com","taxa","em","dólar","?"],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "B-RISCO","I-RISCO","O","O","O","O","B-TAXA_ATRELADA","O"]
    },
    # 15 ─────────────────────────────────────────────────────────────────────
    {
        "tokens": ["Mostre","4","fundos","imobiliários","com","risco","baixo",",","rentabilidade","maior","que",
                   "11","%","e","taxa","atrelada","ao","IPCA","."],
        "ner_tags": ["O","B-QUANTIDADE","B-TIPO_ATIVO","I-TIPO_ATIVO","O",
                     "B-RISCO","I-RISCO","O",
                     "B-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","I-RENTABILIDADE","O",
                     "O","O","O","B-TAXA_ATRELADA","O"]
    },
]


In [6]:
# 1 ─ Labels e mapeamentos  ────────────────────────────────────────────────
labels = [
    "O",
    "B-NOME_ATIVO", "I-NOME_ATIVO",
    "B-TIPO_ATIVO", "I-TIPO_ATIVO",
    "B-RENTABILIDADE", "I-RENTABILIDADE",
    "B-RISCO", "I-RISCO",
    "B-TAXA_ATRELADA", "I-TAXA_ATRELADA",
    "B-QUANTIDADE", "I-QUANTIDADE"
]
label_to_id = {l: i for i, l in enumerate(labels)}
id_to_label = {i: l for l, i in label_to_id.items()}


# 4 ─ Converte tags string → IDs  (para *todos* os exemplos) ───────────────
for ex in chain(training_samples, validation_samples):
    ex["ner_tags"] = [label_to_id[tag] for tag in ex["ner_tags"]]

# 5 ─ Cria Datasets separando treino e validação explicitamente ────────────
train_ds = Dataset.from_list(training_samples)
eval_ds  = Dataset.from_list(validation_samples)

# Agora você segue com tokenização, alinhamento de labels
# e o Trainer exatamente como antes.


In [7]:
train_ds

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 10
})

In [8]:
# 3. Tokenização e alinhamento de labels
model_name = "Babelscape/wikineural-multilingual-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length"
    )

    all_aligned_labels = []

    for batch_idx, orig_label_ids in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=batch_idx)
        aligned_labels = []
        previous_word_idx = None

        for word_idx in word_ids:
            # Se for special token ou índice inválido, marca -100
            if word_idx is None or word_idx >= len(orig_label_ids):
                aligned_labels.append(-100)

            elif word_idx != previous_word_idx:
                # primeiro subtoke → copia ID original
                aligned_labels.append(orig_label_ids[word_idx])

            else:
                # subtokes subsequentes → converte B-XXX em I-XXX
                label_id = orig_label_ids[word_idx]
                label_str = id_to_label[label_id]
                if label_str.startswith("B-"):
                    label_str = "I-" + label_str[2:]
                aligned_labels.append(label_to_id[label_str])

            previous_word_idx = word_idx

        all_aligned_labels.append(aligned_labels)

    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs

# Aplique ao dataset:
train_tokenized = train_ds.map(tokenize_and_align_labels, batched=True)
eval_tokenized  = eval_ds.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 5/5 [00:00<00:00, 837.99 examples/s]


In [10]:
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=2)
    true_labels = [
        [id_to_label[l] for l in label_seq if l != -100]
        for label_seq in labels
    ]
    true_preds = [
        [id_to_label[p] for (p, l) in zip(pred_seq, lab_seq) if l != -100]
        for pred_seq, lab_seq in zip(preds, labels)
    ]
    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds)
    }

In [30]:
training_args = TrainingArguments(
    #output_dir="ner_finetuned/",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=15,
    logging_steps=2,
    save_strategy="epoch",
    logging_strategy="epoch",
)

In [31]:
labels

['O',
 'B-NOME_ATIVO',
 'I-NOME_ATIVO',
 'B-TIPO_ATIVO',
 'I-TIPO_ATIVO',
 'B-RENTABILIDADE',
 'I-RENTABILIDADE',
 'B-RISCO',
 'I-RISCO',
 'B-TAXA_ATRELADA',
 'I-TAXA_ATRELADA',
 'B-QUANTIDADE',
 'I-QUANTIDADE']

In [32]:
# 4. Preparar modelo e Trainer
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(labels), ignore_mismatched_sizes=True
)
model.config.label2id = label_to_id
model.config.id2label = id_to_label
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 5. Fine-tuning
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([13, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\alber\AppData\Local\Temp\ipykernel_13400\1030812318.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnin

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.731500,2.019787,0.413534,0.000000
2,1.974500,1.651506,0.421053,0.000000
3,1.650600,1.378568,0.654135,0.208333
4,1.352700,1.074334,0.721805,0.173913
5,1.132200,0.905179,0.744361,0.222222
6,0.918000,0.734075,0.766917,0.244898
7,0.776900,0.605085,0.879699,0.603774
8,0.661200,0.522474,0.894737,0.654545
9,0.549500,0.494015,0.879699,0.618182
10,0.478100,0.412139,0.917293,0.807692


c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used

TrainOutput(global_step=30, training_loss=0.9268367489178976, metrics={'train_runtime': 907.3984, 'train_samples_per_second': 0.165, 'train_steps_per_second': 0.033, 'total_flos': 39198411417600.0, 'train_loss': 0.9268367489178976, 'epoch': 15.0})

In [33]:
# 6. Avaliação
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)


c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluation metrics: {'eval_loss': 0.321000874042511, 'eval_accuracy': 0.9398496240601504, 'eval_f1': 0.8076923076923077, 'eval_runtime': 2.7065, 'eval_samples_per_second': 1.847, 'eval_steps_per_second': 0.369, 'epoch': 15.0}


In [37]:
# 7. Inferência com pipeline (já configurado anteriormente)
nlp_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    #grouped_entities=True,
    #aggregation_strategy="simple",
)

sample = "Quais são os 5 ativos de renda fixa com risco baixo e rentabilidade acima de 10% ao ano, atrelados ao CDI?"
result = nlp_pipeline(sample)

# Impressão formatada
print(f"{'Token':<20} {'Entidade':<15} {'Score':<6}")
print("-" * 45)
for item in result:
    word  = item["word"]
    ent   = item["entity"]
    score = item["score"]
    print(f"{word:<20} {ent:<15} {score:.2f}")

Device set to use cpu


Token                Entidade        Score 
---------------------------------------------
5                    B-QUANTIDADE    0.68
renda                B-TIPO_ATIVO    0.46
fi                   I-TIPO_ATIVO    0.82
##xa                 I-TIPO_ATIVO    0.82
risco                B-RISCO         0.64
baixo                I-RISCO         0.79
renta                B-RENTABILIDADE 0.86
##bilidade           I-RENTABILIDADE 0.99
acima                I-RENTABILIDADE 0.99
de                   I-RENTABILIDADE 0.99
10                   I-RENTABILIDADE 0.99
%                    I-RENTABILIDADE 0.99
ao                   I-RENTABILIDADE 0.99
ano                  I-RENTABILIDADE 0.99
CD                   B-TAXA_ATRELADA 0.41
##I                  I-TAXA_ATRELADA 0.54


In [29]:
result

[{'entity': 'I-TIPO_ATIVO',
  'score': np.float32(0.26869383),
  'index': 9,
  'word': 'renda',
  'start': 25,
  'end': 30},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.53479916),
  'index': 15,
  'word': 'e',
  'start': 52,
  'end': 53},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.7878231),
  'index': 17,
  'word': '##bilidade',
  'start': 59,
  'end': 67},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.7999744),
  'index': 18,
  'word': 'acima',
  'start': 68,
  'end': 73},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.7905636),
  'index': 19,
  'word': 'de',
  'start': 74,
  'end': 76},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.8242725),
  'index': 20,
  'word': '10',
  'start': 77,
  'end': 79},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.8321622),
  'index': 21,
  'word': '%',
  'start': 79,
  'end': 80},
 {'entity': 'I-RENTABILIDADE',
  'score': np.float32(0.78526443),
  'index': 22,
  'word': 'ao',
  'start': 81,
  '

In [ ]:
# 8. Relatório detalhado
# Converter inferência e rótulos verdadeiros e imprimir classification report
print(classification_report(
    [["B-PER","O","O","B-LOC","O","O"]],
    [[ent["entity"] for ent in result]]
))